In [ ]:
import pandas as pd

from brfinance import CVMAsyncBackend
from brfinance.utils import get_enet_download_url

from datetime import datetime, date


cvm_httpclient = CVMAsyncBackend()

In [ ]:
# Dict de códigos CVM para todas as empresas
cvm_codes = cvm_httpclient.get_cvm_codes()
display(cvm_codes)

# Dict de todas as categorias de busca disponíveis (Fato relevante, DFP, ITR, etc.)
categories = cvm_httpclient.get_consulta_externa_cvm_categories()
display(categories)

# Ditc tipos de participantes
tipo_participante = cvm_httpclient.get_consulta_externa_cvm_tipo_participante()
display(tipo_participante)

In [ ]:
# Realizando busca por Empresa
start_date = date(2021, 1, 1)
end_date = date.today()
cvm_codes_list = [21610] # B3
category = ["EST_3","EST_4","IPE_4_-1_-1"] # Códigos de categoria para DFP, ITR e fatos relevantes
last_ref_date = False # Se "True" retorna apenas o último report no intervalo de datas
participant_type = [1] # Companhia aberta

search_result = cvm_httpclient.get_consulta_externa_cvm_results(
    start_date=start_date,
    end_date=end_date,
    cod_cvm=cvm_codes_list,
    participant_type=participant_type,
    category=category,
    last_ref_date=last_ref_date
    )
display(search_result)

In [ ]:
# Obter demonstrativos

reports_list = [
    'Balanço Patrimonial Ativo',
    'Balanço Patrimonial Passivo',
    'Demonstração do Resultado',
    'Demonstração do Resultado Abrangente',
    'Demonstração do Fluxo de Caixa',
    'Demonstração das Mutações do Patrimônio Líquido',
    'Demonstração de Valor Adicionado'] # Se None retorna todos os demonstrativos disponíveis.

# Filtro search_result para ITR e DFP apenas
search_result = search_result[
    (search_result['categoria']=="DFP - Demonstrações Financeiras Padronizadas") |
    (search_result['categoria']=="ITR - Informações Trimestrais")]

search_result = search_result[pd.to_numeric(search_result['numero_seq_documento'], errors='coerce').notnull()]

for index, row in search_result.iterrows():
    empresa = f"{row['cod_cvm']} - {cvm_codes[row['cod_cvm']]} - {row['numero_seq_documento']} - {row['codigo_tipo_instituicao']}"
    print("*" * 20, empresa, "*" * 20)
    
    reports = cvm_httpclient.get_report(row["numero_seq_documento"], row["codigo_tipo_instituicao"], reports_list=reports_list)
    for report in reports:
        print(report)
        reports[report]["cod_cvm"] = row["cod_cvm"]
        reports[report]["ref_date"] = row["ref_date"]
        reports[report]["numSequencia"] = row["numSequencia"]
        display(reports[report].head())
        